<a href="https://colab.research.google.com/github/ShrutiKharate/K-Means-vs.-Hierarchical-K-Means-clustering-/blob/main/K_Means_Hierarchical-K-Means-clustering_more_robust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Assignment Solution: K-Means vs Hierarchical K-Means (Bisecting) on Iris

- K-Means model selection (k=2..8): **WCSS**, **Silhouette**, (optional **CH/DB**)
- Final K-Means model: plots (**Elbow**, **Silhouette vs k**, **Silhouette plot**, **PCA scatter & centroids**), metrics (**ARI, NMI, V-measure, Homogeneity, Completeness**), confusion matrix, centroids.
- **Hierarchical (Bisecting) K-Means** to 3 leaves: split trace with **ΔSSE**, final clusters, metrics, PCA scatter & centroids, confusion matrix.
- Side-by-side metrics comparison.



In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, silhouette_samples,
    adjusted_rand_score, normalized_mutual_info_score,
    v_measure_score, homogeneity_score, completeness_score,
    calinski_harabasz_score, davies_bouldin_score
)

import os

OUTDIR = "outputs_kmeans_bisect"
os.makedirs(OUTDIR, exist_ok=True)

print("Output directory:", OUTDIR)


Output directory: outputs_kmeans_bisect


In [2]:

iris = load_iris()
X = pd.DataFrame(iris.data, columns=[
    "sepal_length","sepal_width","petal_length","petal_width"
])
y = iris.target
y_names = [iris.target_names[i] for i in y]

scaler = StandardScaler()
Z = scaler.fit_transform(X.values)

print("X shape:", X.shape)
X.head()


X shape: (150, 4)


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:





import numpy.linalg as LA

def sse_for_points(Z_subset):
    """Sum of squared distances to the mean (centroid)."""
    if len(Z_subset) == 0:
        return 0.0
    mu = Z_subset.mean(axis=0, keepdims=True)
    diffs = Z_subset - mu
    return float(np.sum(diffs**2))

def sse_for_labels(Z, labels):
    """Total SSE given labels (0..k-1)."""
    total = 0.0
    for c in np.unique(labels):
        Zc = Z[labels == c]
        total += sse_for_points(Zc)
    return float(total)

def centroid_matrix(Z, labels):
    """Return centroids in standardized feature space sorted by cluster id."""
    Cs = []
    for c in np.unique(labels):
        Cs.append(Z[labels == c].mean(axis=0))
    return np.vstack(Cs)

def plot_elbow(ks, wcss, path):
    plt.figure()
    plt.plot(ks, wcss, marker='o')
    plt.xlabel("k")
    plt.ylabel("WCSS (Inertia)")
    plt.title("K-Means Elbow (WCSS vs k)")
    plt.tight_layout()
    plt.savefig(path, dpi=140)
    plt.close()

def plot_silhouette_vs_k(ks, sils, path):
    plt.figure()
    plt.plot(ks, sils, marker='o')
    plt.xlabel("k")
    plt.ylabel("Mean Silhouette")
    plt.title("K-Means: Silhouette vs k")
    plt.tight_layout()
    plt.savefig(path, dpi=140)
    plt.close()

def plot_pca_scatter_with_centroids(Z, labels, centroids, title, path):
    pca = PCA(n_components=2, random_state=42)
    Z2 = pca.fit_transform(Z)
    C2 = pca.transform(centroids)
    plt.figure()
    for c in np.unique(labels):
        mask = (labels == c)
        plt.scatter(Z2[mask,0], Z2[mask,1], label=f"Cluster {c}", alpha=0.8)
    plt.scatter(C2[:,0], C2[:,1], marker='X', s=120, label="Centroids")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

def silhouette_plot(Z, labels, path):
    # Overall silhouette values
    s_vals = silhouette_samples(Z, labels)
    # Arrange by cluster then by value
    order = np.argsort(labels*1e6 + (1 - s_vals))  # group by cluster, then descending s
    s_sorted = s_vals[order]
    l_sorted = labels[order]
    plt.figure(figsize=(6, 5))
    y_pos = 0
    for c in np.unique(l_sorted):
        sc = s_sorted[l_sorted == c]
        x = np.arange(len(sc))
        plt.barh(y_pos + x, sc)
        y_pos += len(sc) + 5  # gap between clusters
    plt.axvline(np.mean(s_vals), linestyle='--')
    plt.xlabel("Silhouette value")
    plt.ylabel("Samples (grouped by cluster)")
    plt.title("Silhouette Plot (Final K-Means)")
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

def confusion_df(pred, true, true_names=None):
    if true_names is None:
        true_names = true
    df = pd.crosstab(pd.Series(pred, name="Cluster"),
                     pd.Series(true_names, name="Species"))
    return df

def plot_confusion_heatmap(df, title, path):
    plt.figure(figsize=(5.5, 4.3))
    plt.imshow(df.values, aspect='auto')
    plt.xticks(range(df.shape[1]), df.columns, rotation=45, ha='right')
    plt.yticks(range(df.shape[0]), df.index)
    plt.title(title)
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            plt.text(j, i, str(df.values[i, j]), ha='center', va='center')
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

def metrics_dict(Z, labels, y_true):
    out = {
        "ARI": adjusted_rand_score(y_true, labels),
        "NMI": normalized_mutual_info_score(y_true, labels),
        "V_measure": v_measure_score(y_true, labels),
        "Homogeneity": homogeneity_score(y_true, labels),
        "Completeness": completeness_score(y_true, labels),
        "Silhouette": silhouette_score(Z, labels)
    }
    return out

def bisecting_kmeans(Z, K=3, n_init=10, random_state=42, max_trials=5):
    """
    Top-down bisection until K clusters.
    At each step, pick cluster with max SSE and split it with k=2 KMeans.
    Keep the split with best ΔSSE among max_trials random states.
    Returns: labels (0..K-1), centroids (K x p), trace (list of dicts)
    """
    # Start: one cluster containing all indices
    clusters = [np.arange(Z.shape[0])]
    trace = []
    rng = np.random.RandomState(random_state)

    def cluster_sse(idx):
        return sse_for_points(Z[idx])

    while len(clusters) < K:
        # Choose cluster with max SSE
        sse_list = [cluster_sse(idx) for idx in clusters]
        parent_i = int(np.argmax(sse_list))
        parent_idx = clusters[parent_i]
        parent_sse = sse_list[parent_i]

        # Split parent with multiple trials; keep best ΔSSE
        best = None
        for t in range(max_trials):
            km = KMeans(n_clusters=2, init="k-means++", n_init=n_init,
                        random_state=rng.randint(0, 10_000))
            sub_labels = km.fit_predict(Z[parent_idx])
            child1 = parent_idx[sub_labels == 0]
            child2 = parent_idx[sub_labels == 1]
            child_sse_1 = cluster_sse(child1)
            child_sse_2 = cluster_sse(child2)
            delta = parent_sse - (child_sse_1 + child_sse_2)
            if (best is None) or (delta > best["delta"]):
                best = {
                    "child1": child1, "child2": child2,
                    "child_sse_1": child_sse_1, "child_sse_2": child_sse_2,
                    "delta": delta
                }

        # Replace parent with the two children
        clusters.pop(parent_i)
        clusters.append(best["child1"])
        clusters.append(best["child2"])

        trace.append({
            "parent_index": parent_i,
            "parent_sse": parent_sse,
            "child_sse_1": best["child_sse_1"],
            "child_sse_2": best["child_sse_2"],
            "delta_sse": best["delta"]
        })

    # Build global labels 0..K-1
    labels = np.empty(Z.shape[0], dtype=int)
    for cid, idx in enumerate(clusters):
        labels[idx] = cid

    C = centroid_matrix(Z, labels)
    return labels, C, pd.DataFrame(trace)


In [4]:

ks = list(range(2, 9))
wcss = []
sil_means = []
ch_scores = []
db_scores = []

for k in ks:
    km = KMeans(n_clusters=k, init="k-means++", n_init=10, random_state=42)
    labels = km.fit_predict(Z)
    wcss.append(float(km.inertia_))
    sil_means.append(float(silhouette_score(Z, labels)))
    ch_scores.append(float(calinski_harabasz_score(Z, labels)))
    db_scores.append(float(davies_bouldin_score(Z, labels)))

sel_tbl = pd.DataFrame({
    "k": ks,
    "WCSS": wcss,
    "Silhouette": sil_means,
    "CH": ch_scores,
    "DB": db_scores
})
sel_tbl_path = os.path.join(OUTDIR, "kmeans_model_selection.csv")
sel_tbl.to_csv(sel_tbl_path, index=False)

print(sel_tbl)

# Plots
plot_elbow(ks, wcss, os.path.join(OUTDIR, "kmeans_elbow.png"))
plot_silhouette_vs_k(ks, sil_means, os.path.join(OUTDIR, "kmeans_silhouette_vs_k.png"))

# Pick final k by max silhouette (tie-break by lower WCSS)
best_sil = max(sil_means)
candidates = [k for k, s in zip(ks, sil_means) if s == best_sil]
if len(candidates) > 1:
    k_final = min(candidates, key=lambda kk: wcss[ks.index(kk)])
else:
    k_final = candidates[0]

print("Chosen k (by silhouette, tie-break WCSS):", k_final)


   k        WCSS  Silhouette          CH        DB
0  2  222.361705    0.581750  251.349339  0.593313
1  3  139.820496    0.459948  241.904402  0.833595
2  4  114.092547    0.386941  207.265914  0.869814
3  5   90.927514    0.345901  202.951525  0.948317
4  6   81.544391    0.317079  183.109118  1.053677
5  7   72.631144    0.320197  173.051904  0.990528
6  8   62.540606    0.338692  174.330702  0.914982
Chosen k (by silhouette, tie-break WCSS): 2


In [5]:

# Fit final K-Means
km_final = KMeans(n_clusters=k_final, init="k-means++", n_init=20, random_state=42)
labels_km = km_final.fit_predict(Z)
C_km = km_final.cluster_centers_
wcss_final = km_final.inertia_

# Metrics
m_km = metrics_dict(Z, labels_km, y)
metrics_km = pd.DataFrame([m_km], index=[f"KMeans_k{k_final}"])
metrics_km_path = os.path.join(OUTDIR, "metrics_kmeans.csv")
metrics_km.to_csv(metrics_km_path)

# PCA scatter + centroids
plot_pca_scatter_with_centroids(Z, labels_km, C_km,
    title=f"K-Means (k={k_final}) — PC1–PC2",
    path=os.path.join(OUTDIR, "kmeans_pca_scatter.png"))

# Silhouette plot
silhouette_plot(Z, labels_km, os.path.join(OUTDIR, "kmeans_silhouette_plot.png"))

# Confusion matrix vs species
conf_km = confusion_df(labels_km, y, y_names)
conf_km_path = os.path.join(OUTDIR, "confusion_kmeans.csv")
conf_km.to_csv(conf_km_path)
plot_confusion_heatmap(conf_km, f"K-Means (k={k_final}) — Clusters vs Species",
                       os.path.join(OUTDIR, "kmeans_confusion.png"))

# Save centroids (standardized and original units)
centroids_std = pd.DataFrame(C_km, columns=X.columns)
centroids_std.to_csv(os.path.join(OUTDIR, "kmeans_centroids_standardized.csv"), index_label="Cluster")

orig_means = X.mean().values
orig_stds  = X.std(ddof=0).values
C_km_orig = C_km * orig_stds + orig_means
centroids_orig = pd.DataFrame(C_km_orig, columns=X.columns)
centroids_orig.to_csv(os.path.join(OUTDIR, "kmeans_centroids_original_units.csv"), index_label="Cluster")

print("K-Means metrics:\n", metrics_km)
print("\nK-Means centroids (original units):\n", centroids_orig)


K-Means metrics:
                 ARI      NMI  V_measure  Homogeneity  Completeness  Silhouette
KMeans_k2  0.568116  0.73368    0.73368      0.57938           1.0     0.58175

K-Means centroids (original units):
    sepal_length  sepal_width  petal_length  petal_width
0         6.262        2.872         4.906        1.676
1         5.006        3.428         1.462        0.246


In [6]:

K_target = 3
labels_bi, C_bi, trace = bisecting_kmeans(Z, K=K_target, n_init=10, random_state=42, max_trials=8)

# Save trace
trace_path = os.path.join(OUTDIR, "bisect_trace.csv")
trace.to_csv(trace_path, index=False)

# Metrics
m_bi = metrics_dict(Z, labels_bi, y)
metrics_bi = pd.DataFrame([m_bi], index=[f"BisectingKMeans_K{K_target}"])
metrics_bi_path = os.path.join(OUTDIR, "metrics_bisect.csv")
metrics_bi.to_csv(metrics_bi_path)

# PCA scatter + centroids
plot_pca_scatter_with_centroids(Z, labels_bi, C_bi,
    title=f"Bisecting K-Means (K={K_target}) — PC1–PC2",
    path=os.path.join(OUTDIR, "bisect_pca_scatter.png"))

# Confusion matrix
conf_bi = confusion_df(labels_bi, y, y_names)
conf_bi_path = os.path.join(OUTDIR, "confusion_bisect.csv")
conf_bi.to_csv(conf_bi_path)
plot_confusion_heatmap(conf_bi, f"Bisecting K-Means (K={K_target}) — Clusters vs Species",
                       os.path.join(OUTDIR, "bisect_confusion.png"))

# Save centroids (std & original units)
centroids_bi_std = pd.DataFrame(C_bi, columns=X.columns)
centroids_bi_std.to_csv(os.path.join(OUTDIR, "bisect_centroids_standardized.csv"), index_label="Leaf")

C_bi_orig = C_bi * orig_stds + orig_means
centroids_bi_orig = pd.DataFrame(C_bi_orig, columns=X.columns)
centroids_bi_orig.to_csv(os.path.join(OUTDIR, "bisect_centroids_original_units.csv"), index_label="Leaf")

print("Bisecting metrics:\n", metrics_bi)
print("\nBisecting centroids (original units):\n", centroids_bi_orig.head())
print("\nBisect split trace:\n", trace)


Bisecting metrics:
                          ARI       NMI  V_measure  Homogeneity  Completeness  \
BisectingKMeans_K3  0.620135  0.659487   0.659487     0.659127      0.659848   

                    Silhouette  
BisectingKMeans_K3    0.459948  

Bisecting centroids (original units):
    sepal_length  sepal_width  petal_length  petal_width
0      5.006000     3.428000      1.462000     0.246000
1      6.780851     3.095745      5.510638     1.972340
2      5.801887     2.673585      4.369811     1.413208

Bisect split trace:
    parent_index  parent_sse  child_sse_1  child_sse_2   delta_sse
0             0  600.000000   174.693294    47.668411  377.638295
1             0  174.693294    47.768652    44.383434   82.541209


In [7]:

metrics_both = pd.concat([metrics_km, metrics_bi], axis=0)
metrics_both_path = os.path.join(OUTDIR, "metrics_comparison.csv")
metrics_both.to_csv(metrics_both_path)

print(metrics_both)


                         ARI       NMI  V_measure  Homogeneity  Completeness  \
KMeans_k2           0.568116  0.733680   0.733680     0.579380      1.000000   
BisectingKMeans_K3  0.620135  0.659487   0.659487     0.659127      0.659848   

                    Silhouette  
KMeans_k2             0.581750  
BisectingKMeans_K3    0.459948  



## Saved Artifacts (in `outputs_kmeans_bisect/`)
- **Model selection**: `kmeans_model_selection.csv`, `kmeans_elbow.png`, `kmeans_silhouette_vs_k.png`
- **K-Means (final)**: `kmeans_centroids_standardized.csv`, `kmeans_centroids_original_units.csv`, `kmeans_confusion.png`, `kmeans_confusion.csv`, `kmeans_pca_scatter.png`, `kmeans_silhouette_plot.png`, `metrics_kmeans.csv`
- **Bisecting K-Means**: `bisect_trace.csv`, `bisect_centroids_standardized.csv`, `bisect_centroids_original_units.csv`, `bisect_confusion.png`, `bisect_confusion.csv`, `bisect_pca_scatter.png`, `metrics_bisect.csv`
- **Comparison**: `metrics_comparison.csv`
